## Загрузка датасета на HuggingFace

Этот ноутбук дает пример того, как залить локальный датасет на ХФ. Адаптируйте его под свой датасет. Затем, выложите в гитхаб получившийся ноутбук (приложите к своему датасету), чтобы всегда был доступен код для заливки вашего датасета на ХФ. Убедитесь, что ячейки последовательно запускаются.

In [42]:
from PIL import Image
import json
import datasets
from tqdm import tqdm
import pandas as pd
from datasets import Dataset, Features, Value

import os

### Подготовка данных

#### WARNING! 

Если ваш датасет является __ПРИВАТНЫМ__, то оставьте `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS` равным `True`. Иначе, поставьте `False`. Этот флаг дальше используется, чтобы стереть ответы перед загрузкой на ХФ датасета. На ХФ даже приватно не должно лежать датасетов с ответами!

In [2]:
MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS = True

Данный пример рассчитан на загрузку на ХФ локального датасета.

Параметр `path_to_data` - это путь ДО файлов `shots.json` и `test.json`, которые вы будете дальше загружать на ХФ в виде датасета или домена датасета. 

Параметр `path_to_meta` - это путь ДО меты датасета.

Итоговые пути будут собираться из `path_to_data` / `path_to_meta` + `file_name.json`!

In [129]:
task_name = "aqua_bench"

In [130]:
path_to_train = f"./../industrial_tasks/{task_name}/data/train.jsonl"
path_to_data = f"./../industrial_tasks/{task_name}/data/test.jsonl"

#### Подгрузка данных

Считайте сплиты и мету датасета (домена датасета). Это просто JSON файлики либо внутри прямо папки датасета, либо внутри папки по названию домена, который вы будете загружать.

In [131]:
test = pd.read_json(path_to_data, lines=True)
train = pd.read_json(path_to_train, lines=True)

На ХФ вы загружаете датасет, где у КАЖДОГО сэмпла вместо числа в поле instruction стоит промпт. Число указывает, какой по индексу взять промпт из секции с промптами в мете датасета.

In [86]:
for card in shots:
    card["instruction"] = prompts[card["instruction"]]

for card in test:
    card["instruction"] = prompts[card["instruction"]]

NameError: name 'shots' is not defined

Теперь вам нужно "обработать" вашу модальность(-и). Если у вас в датасете картинки, то для каждой вы вместо пути к картинке подгружаете саму картинку и убираете у нее `filename`.

Зачем убирать `filename`? Если этого не сделать, то потом при конвертации в байты модуль datasets увидит, что картинка взята по какому-то пути и не будет ее конвертировать в байты. А нам нужно превратить PIL.Image в байткод и загрузить на ХФ именно его.

Для картинок вы можете воспользоваться функцией `convert_images`. Она принимает на вход: 
- `dataset_split`: список словарей, это список сэмплов сплита (json файлик, который вы загрузили ранее)
- `feature_name`: строка, название поля, которое преобразуется (например, "image", "image_1")
- `path_to_samples`: строка. Если ваши коллеги для картинок указывали не полный путь до них, а только название файлика, то, чтобы открыть файлик, вам придется указать, как попасть в папку samples, где и находятся картинки. Тогда, укажите в `path_to_samples` путь от папки, где у вас этот ноутбук до папки `samples` (включая ее в путь)

In [ ]:
def convert_images(
    dataset_split: list[dict], feature_name: str, path_to_samples: str = None
):
    for card in tqdm(dataset_split):
        if path_to_samples is None:
            path_to_image = card["inputs"][feature_name]
        else:
            path_to_image = os.path.join(path_to_samples, card["inputs"][feature_name])

        card["inputs"][feature_name] = Image.open(path_to_image)
        card["inputs"][feature_name].filename = ""

In [ ]:
convert_images(shots, "image")
convert_images(test, "image")

Для аудио модальности файл ничем не открываем, а просто сохраняем ссылки на аудио-данные. Обработка аудио-данных будет сделана позднее с помощью datasets.Audio() при формировании features.

In [ ]:
def convert_audio(
    dataset_split: list[dict], feature_name: str, path_to_samples: str = None
):
    for card in tqdm(dataset_split):
        if path_to_samples is None:
            path_to_audio = card["inputs"][feature_name]
        else:
            path_to_audio = os.path.join(path_to_samples, card["inputs"][feature_name])

        card["inputs"][feature_name] = path_to_audio


convert_audio(shots, "audio")
convert_audio(test, "audio")

#### Убираем ответы для приватных задач

Надеемся, вы поставили в начале ноутбука корректное значение `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS`.

Если там стоит `True`, то в `test` сплите ответы на все задания стираются. Вместо них остается пустая строка, чтобы вы случайно не пушнули на ХФ датасет с заполненными ответами, и они не утекли.

In [100]:
def hide_answers(dataset):
    for ind, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
        dataset.at[ind, "outputs"] = ""

In [132]:
if MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS:
    hide_answers(test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 992/992 [00:00<00:00, 10795.76it/s]


In [133]:
test.head(1)

,instruction,inputs,outputs,meta
0,Ниже приведены вопросы с множественным выбором...,{'text': 'Какие приемы используют для дезинфек...,,"{'id': 227, 'domain': 'Индустриальная аквакуль..."


In [134]:
train.head(1)

,instruction,inputs,outputs,meta
0,Ниже приведены вопросы с множественным выбором...,{'text': 'Какие пути интенсификации использова...,"A, B","{'id': 25, 'domain': 'Индустриальная аквакульт..."


### Создаем датасет для загрузки на ХФ

#### Аннотация полей датасета

В `features` повторяется структура КАЖДОГО сэмпла вашего датасета с описанием формата данных в каждом поле. 
- `instruction` всегда строка
- `meta` - id всегда целое число

Далее смотрите по тому, какие поля у вашего датасета.

`features` нужен для того, чтобы ХФ сам автоматически создал техническую часть README.md датасета, заполнив ее информацией, которая используется при загрузке датасета. Отсутствие `features` может и обычно приводит к невозможности использовать датасет. Ровно такие же последствия будут от ошибок в заполнении (например, неправильно указан тип данных).

__Внимание!__ Если у вас в датасете в разных вопросах разное количество ответов, то поля в `features` нужно заполнить для сэмпла с НАИБОЛЬШИМ количеством ответов. Иначе говоря, представьте, что у вас у всех вопросов в датасете максимальное количество вариантов ответа, просто некоторые пустые. Вот из такого соображения и заполняйте `features`. Он один на весь датасет и должен охватывать все поля, которые в нем встречаются!

In [135]:
features = Features(
    {
        "instruction": Value("string"),
        "inputs": {
            "text": Value("string"),
            "option_a": Value("string"),
            "option_b": Value("string"),
            "option_c": Value("string"),
            "option_d": Value("string"),
            "option_e": Value("string"),
            "option_f": Value("string"),
            "option_g": Value("string"),
            "option_h": Value("string"),
        },
        "outputs": Value("string"),
        "meta": {
            "id": Value("int32"),
            "domain": Value("string"),
        },
    }
)

#### Создание датасетов для каждого сплита

Теперь создаем сплиты датасета. Можно это сделать либо в одну строку:

In [136]:
ds_test = Dataset.from_pandas(
    test,
    preserve_index=False,  # don’t add the pandas index as a column :contentReference[oaicite:1]{index=1}
    features=features,
)

ds_train = Dataset.from_pandas(train, preserve_index=False, features=features)

In [125]:
# agro bench
ds_test, ds_train, ds_test[0], ds_train[0]

(Dataset({
     features: ['instruction', 'inputs', 'outputs', 'meta'],
     num_rows: 2642
 }),
 Dataset({
     features: ['instruction', 'inputs', 'outputs', 'meta'],
     num_rows: 293
 }),
 {'instruction': 'Ниже приведены вопросы с множественным выбором (с ответами) по теме Ботаника. Напиши только букву/буквы ответа.\n\n{{text}}\n\nВыбери ответ из вариантов:\nA. {{option_a}}\nB. {{option_b}}\nC. {{option_c}}\nD. {{option_d}}\nОтвет:',
  'inputs': {'text': 'Какое из перечисленных утверждений о семени покрытосеменных является неверным?',
   'option_a': 'семя образуется из семязачатка',
   'option_b': 'семя состоит из интегументов и ядра',
   'option_c': 'семя находится в завязи цветка',
   'option_d': 'семя имеет запасную питательную ткань',
   'option_e': None,
   'option_f': None,
   'option_g': None,
   'option_h': None},
  'outputs': '',
  'meta': {'id': 0, 'domain': 'Ботаника'}},
 {'instruction': 'Ниже приведены вопросы с множественным выбором (с ответами) по теме Ботаника. Напи

In [137]:
# aqua bench
ds_test, ds_train, ds_test[0], ds_train[0]

(Dataset({
     features: ['instruction', 'inputs', 'outputs', 'meta'],
     num_rows: 992
 }),
 Dataset({
     features: ['instruction', 'inputs', 'outputs', 'meta'],
     num_rows: 110
 }),
 {'instruction': 'Ниже приведены вопросы с множественным выбором (с ответами) по теме Индустриальная аквакультура. Напиши только букву/буквы ответа.\n\n{{text}}\n\nВыбери ответ из вариантов:\nA. {{option_a}}\nB. {{option_b}}\nC. {{option_c}}\nD. {{option_d}}\nE. {{option_e}}\nОтвет:',
  'inputs': {'text': 'Какие приемы используют для дезинфекции воды в рыбоводных хозяйствах?',
   'option_a': 'диспергирование',
   'option_b': 'ультрафиолетовое облучение',
   'option_c': 'озонирование',
   'option_d': 'обработка серебром',
   'option_e': 'хлорирование',
   'option_f': None,
   'option_g': None,
   'option_h': None},
  'outputs': '',
  'meta': {'id': 227, 'domain': 'Индустриальная аквакультура'}},
 {'instruction': 'Ниже приведены вопросы с множественным выбором (с ответами) по теме Индустриальная акв

### Загрузка датасета на ХФ

Для загрузки на ХФ вам понадобятся:
- Токен. Это строка, содержащая ключик, который позволит вам записывать в репозиторий. 
- Путь для записи. Это тоже строка, которая содержит путь, по которому вы выложите свой датасет. Этот путь содержит название аккаунта (MERA-evaluation) и название вашего датасета. Название датасета пишите ровно так, как оно заявлено в мете! Регистр тоже имеет значение!

Советуем опубликовывать сперва всё приватно, и выслать на почту mera@a-ai.ru токен и путь для верификации. 
Если ваш сет публичный и вы хотите отправить всё публично, то в Merge request просто пришлите путь к сету.

In [111]:
### TOKEN
token = ""
###

### UPLOAD PATH
dataset_path_hub = "MERA-evaluation/ruTXTAgroBench"
dataset_path_hub = "MERA-evaluation/ruTXTAquaBench"
dataset_path_hub = "MERA-evaluation/ruTXTMedQFundamental"
###

In [138]:
dataset_path_hub = "MERA-evaluation/ruTXTAquaBench"
ds_test.push_to_hub(
    dataset_path_hub, split="test", private=True, token=token
)  # опубликовать приватно
ds_train.push_to_hub(
    dataset_path_hub, split="train", private=True, token=token
)  # опубликовать приватно

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/807 [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/MERA-evaluation/ruTXTAquaBench/commit/48d68ac46129cbbea32acc6a440a88d1ef583b22', commit_message='Upload dataset', commit_description='', oid='48d68ac46129cbbea32acc6a440a88d1ef583b22', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/MERA-evaluation/ruTXTAquaBench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='MERA-evaluation/ruTXTAquaBench'), pr_revision=None, pr_num=None)

In [126]:
dataset_path_hub = "MERA-evaluation/ruTXTAgroBench"

ds_test.push_to_hub(
    dataset_path_hub, split="test", private=True, token=token
)  # опубликовать приватно
ds_train.push_to_hub(
    dataset_path_hub, split="train", private=True, token=token
)  # опубликовать приватно

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/810 [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/910 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/MERA-evaluation/ruTXTAgroBench/commit/fa86f44730d0cd4b930ca220ae0a823d51904b6b', commit_message='Upload dataset', commit_description='', oid='fa86f44730d0cd4b930ca220ae0a823d51904b6b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/MERA-evaluation/ruTXTAgroBench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='MERA-evaluation/ruTXTAgroBench'), pr_revision=None, pr_num=None)

In [ ]:
dataset_path_hub = "MERA-evaluation/ruTXTMedQFundamental"

ds_test.push_to_hub(
    dataset_path_hub, split="test", private=True, token=token
)  # опубликовать приватно
ds_train.push_to_hub(
    dataset_path_hub, split="train", private=True, token=token
)  # опубликовать приватно

### Проверка того, как датасет загрузился на ХФ

После загрузки датасета будет полезно посмотреть, как его будет видеть любой человек, который после вашей загрузки его скачает. 

Загрузите датасет целиком, используя `datasets.load_dataset(dataset_path_hub)`, а затем проверьте, что:
- все поля на месте. Если у вас в датасете у разных вопросов было разное количество вариантов ответа, то теперь их везде станет одинаковое количество. Недостающие варианты ответа у каждого вопроса теперь будут прописаны, но будут иметь значение `None`. Это нормально.
- ваша модальность корректно обработалась. Если у вас в датасете были картинки, то все они должны превратиться в байткод. Не должно остаться ни одной картинки, которая не конвертирована в байты. Если у картинки есть и байты, и путь прописан (а не `None`), то это окей. `bytes` точно должны быть заполнены, `path` может быть None.
- датасет идентичен по содержанию исходному. То есть, в исходном JSON и загруженном датасете вопрос с одинаковым `id` имеет одинаково заполненные поля (кроме тех, что заполняются `None`, как описано выше).

In [113]:
ds = datasets.load_dataset(dataset_path_hub)

README.md:   0%|          | 0.00/908 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/106k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/510 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4590 [00:00<?, ? examples/s]

In [114]:
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 510
    })
    test: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 4590
    })
})

In [116]:
ds["train"][0]

{'instruction': 'Ты - опытный врач-терапевт. Ты проводишь прием пациента.\nТебе будут задаваться вопросы с вариантами ответов по твоей специальности.\nТвоя задача - выбрать один фактологически верный вариант ответа!\nВажно: в своем ответе верни только одну букву A, B, C или D!\n{text}\n\nВыбери ответ из вариантов:\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nОтвет:',
 'inputs': {'text': 'На какой неделе появляются сомиты?',
  'option_a': 'A. на 6 неделе',
  'option_b': 'B. на 5 неделе',
  'option_c': 'C. на 4 неделе',
  'option_d': 'D. на 3 неделе',
  'option_e': None,
  'option_f': None,
  'option_g': None,
  'option_h': None},
 'outputs': 'C',
 'meta': {'id': 0, 'domain': 'anatomy'}}

In [115]:
ds["test"][0]

{'instruction': 'Ты - опытный врач-терапевт. Ты проводишь прием пациента.\nТебе будут задаваться вопросы с вариантами ответов по твоей специальности.\nТвоя задача - выбрать один фактологически верный вариант ответа!\nВажно: в своем ответе верни только одну букву A, B, C или D!\n{text}\n\nВыбери ответ из вариантов:\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nОтвет:',
 'inputs': {'text': 'Как называется одна из частей мужского мочеиспускательного канала?',
  'option_a': 'A. губчатая часть',
  'option_b': 'B. промежуточная часть',
  'option_c': 'C. пещеристая часть',
  'option_d': 'D. семенная часть',
  'option_e': None,
  'option_f': None,
  'option_g': None,
  'option_h': None},
 'outputs': '',
 'meta': {'id': 0, 'domain': 'anatomy'}}

In [128]:
# agro
ds = datasets.load_dataset(dataset_path_hub)
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 293
    })
    test: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 2642
    })
})

In [139]:
# aqua
ds = datasets.load_dataset(dataset_path_hub)
ds

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/36.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/198k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/992 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 110
    })
    test: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 992
    })
})